In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
import networkx as nx
from scipy.spatial.distance import cdist
import sklearn
from sklearn.cluster import Birch, KMeans, MeanShift, DBSCAN, SpectralClustering
from sklearn.decomposition import TruncatedSVD
from normalized_cut import normalized_cut

from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points
from aggregate_pointcloud import aggregate_pointcloud
from reproject_merged_pointcloud import reproject_points_to_label, merge_associations, merge_features
from visualization_utils import generate_random_colors
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 7

config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(3, 25),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

Now we aggregate a large point cloud based on (ind_start, ind_end)

In [ ]:
ind_start = 0
ind_end = 100

pcd, T_pcd, all_poses = aggregate_pointcloud(dataset, ind_start, ind_end, clip_to_imageframe=False, return_poses=True)
first_position = all_poses[0][:3,3]

num_points = np.asarray(pcd.points).shape[0]
print("num points: ", num_points)

Now we subsample the poses based on a voxel_size

In [ ]:
all_positions = []
for p in all_poses:
    all_positions.append(tuple(p[:3,3]))

sampled_indices_local = list(subsample_positions(all_positions, voxel_size=1))
sampled_indices_global = list(subsample_positions(all_positions, voxel_size=1) + ind_start)

poses = np.array(all_poses)[sampled_indices_local]
positions = np.array(all_positions)[sampled_indices_local]

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
chunk_size = np.array([13, 13, 13]) #meters
overlap = 3 #meters

pcd_chunks, center_positions, center_ids = chunks_from_pointcloud(pcd, T_pcd, positions, poses, first_position, 
                                                                  sampled_indices_global, chunk_size, overlap)

pcd_chunks_minor_downsampling = []
pcd_chunks_major_downsampling = []
minor_voxel_size = 0.1
major_voxel_size = 0.35

for chunk in pcd_chunks:
    pcd_chunks_minor_downsampling.append(chunk.voxel_down_sample(voxel_size=minor_voxel_size))
    pcd_chunks_major_downsampling.append(chunk.voxel_down_sample(voxel_size=major_voxel_size))
    print("Downsampled from", np.asarray(chunk.points).shape, "to", np.asarray(pcd_chunks_minor_downsampling[-1].points).shape, "points (minor downsampling)",
         "and to", np.asarray(pcd_chunks_major_downsampling[-1].points).shape, "points (major downsampling)")

Now we iterate over the chunks, load the features and perform the normalized cuts

In [ ]:
patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)

for sequence in range(len(center_ids)):

    print("Start of sequence ", sequence)

    first_id = patchwise_indices[sequence][0]
    center_id = center_ids[sequence]
    center_position = center_positions[sequence]

    pcd_chunk = pcd_chunks[sequence]
    chunk_minor = pcd_chunks_minor_downsampling[sequence]
    chunk_major = pcd_chunks_major_downsampling[sequence]

    points_major = np.asarray(chunk_major.points)
    num_points_major = points_major.shape[0]  


    print(num_points_major, "points in downsampled chunk (major)")

    tarl_features = tarl_features_per_patch(dataset, chunk_major, center_id, T_pcd, center_position,
                                            sampled_indices_global, chunk_size, major_voxel_size)

    cams = ["cam2", "cam3"]
    sam_reprojections, dinov2_reprojections, hpr_masks = image_based_features_per_patch(dataset, chunk_minor, T_pcd, sampled_indices_global, 
                                                                                        first_id, cams, hpr_radius = 10000, cam_id=0, return_hpr_masks=True)
    
    sam_features_minor = merge_associations(sam_reprojections, len(chunk_minor.points))
    dinov2_features_minor = merge_features(dinov2_reprojections, len(chunk_minor.points))
    
    sam_features_major = -1 * np.ones((num_points_major, sam_features_minor.shape[1]))
    dinov2_features_major = np.zeros((num_points_major, dinov2_features_minor.shape[1])) 

    sam_features_major = kDTree_1NN_feature_reprojection(sam_features_major, chunk_major, sam_features_minor, chunk_minor)
    dinov2_features_major = kDTree_1NN_feature_reprojection(dinov2_features_major, chunk_major, dinov2_features_minor, chunk_minor)

    zero_rows = np.sum(~np.array(sam_features_major).any(1))
    ratio = zero_rows / num_points_major

    if ratio > 0.3:
        print("The ratio of points without image-based features is", ratio, ". Skipping this chunk.")
        continue

    spatial_distance = cdist(points_major, points_major)
    dinov2_distance = cdist(dinov2_features_major, dinov2_features_major)
    tarl_distance = cdist(tarl_features, tarl_features)

    proximity_threshold = 2 # meters that points can be apart from each other and still be considered neighbors
    alpha = 0.0 # weight of the spatial proximity term 
    beta = 10.0 # weight of the label similarity term
    gamma = 0.0 # weight of the dinov2 feature similarity term
    theta = 0.0 # weight of the tarl feature similarity term

    sam_edge_weights, mask = sam_label_distance(sam_features_major, spatial_distance, proximity_threshold, beta)
    spatial_edge_weights = mask * np.exp(-alpha * spatial_distance)
    dinov2_edge_weights = mask * np.exp(-gamma * dinov2_distance)
    tarl_edge_weights = mask * np.exp(-theta * tarl_distance)

    A = spatial_edge_weights * sam_edge_weights * dinov2_edge_weights * tarl_edge_weights
    print("Adjacency Matrix built")

    # Remove isolated points
    chunk_major, A = remove_isolated_points(chunk_major, A)
    print(num_points_major - np.asarray(chunk_major.points).shape[0], "isolated points removed")
    num_points_major = np.asarray(chunk_major.points).shape[0]

    num_groups = 0
    norm_cut_threshold = 0.3
    num_iterations = 0
    top3_ratio = 1
    min_groups = 60
    max_groups = 250
    max_top3 = 0.6

    while((num_groups < min_groups or num_groups > max_groups or (top3_ratio > max_top3 and num_groups < max_groups)) and (num_iterations < 5)):

        print("Start of normalized Cuts. Iteration", num_iterations, "with threshold", norm_cut_threshold)
        grouped_labels = normalized_cut(A, np.arange(num_points_major), T = norm_cut_threshold)
        num_groups = len(grouped_labels)
        print("There are", num_groups, "cut regions")

        sorted_groups = sorted(grouped_labels, key=lambda x: len(x))
        num_points_top3 = np.sum([len(g) for g in sorted_groups[-3:]])
        top3_ratio = num_points_top3 / num_points_major
        print("Ratio of points in top 3 groups:", top3_ratio)

        if num_groups > max_groups and top3_ratio > max_top3:
            break
        if num_groups < min_groups or (top3_ratio > max_top3 and num_groups < max_groups):
            norm_cut_threshold *= max_groups / num_groups
            num_iterations += 1
        elif num_groups > max_groups:
            norm_cut_threshold *= min_groups / num_groups
            num_iterations += 1

    if top3_ratio < max_top3:

        random_colors = generate_random_colors(600)

        pcd_color = np.zeros((num_points_major, 3))

        for i, s in enumerate(grouped_labels):
            for j in s:
                pcd_color[j] = np.array(random_colors[i]) / 255

        pcd_chunk.paint_uniform_color([0, 0, 0])
        colors = kDTree_1NN_feature_reprojection(np.asarray(pcd_chunk.colors), pcd_chunk, pcd_color, chunk_major)
        pcd_chunk.colors = o3d.utility.Vector3dVector(colors)

        index_file = str(center_id).zfill(6) + '.pcd'
        file = os.path.join("test_data", index_file)
        o3d.io.write_point_cloud(file, pcd_chunk, write_ascii=False, compressed=False, print_progress=False)
        
        print("Pointcloud written to file")
        print("End of sequence ", sequence)

    else:
        
        print("Too many points in top 3 groups. Still saving for debugging reasons...")

        random_colors = generate_random_colors(600)

        pcd_color = np.zeros((num_points_major, 3))

        for i, s in enumerate(grouped_labels):
            for j in s:
                pcd_color[j] = np.array(random_colors[i]) / 255

        pcd_chunk.paint_uniform_color([0, 0, 0])
        colors = kDTree_1NN_feature_reprojection(np.asarray(pcd_chunk.colors), pcd_chunk, pcd_color, chunk_major)
        pcd_chunk.colors = o3d.utility.Vector3dVector(colors)

        index_file = str(center_id).zfill(6) + '_deleted.pcd'
        file = os.path.join("test_data", index_file)
        o3d.io.write_point_cloud(file, pcd_chunk, write_ascii=False, compressed=False, print_progress=False)
        
        print("Pointcloud written to file")
        print("End of sequence ", sequence)
        continue